In [6]:
import pandas as pd
from datetime import datetime, date

In [32]:
df = pd.read_csv('../data/transactions.csv', parse_dates=['Date'])

In [33]:
df = df.loc[df['Date'] >= datetime(2021, 1, 1)]

In [34]:
df = df.set_index('Date')

In [37]:
monthly = round(df.resample('M')[['Amount']].sum())

In [56]:
monthly.index = monthly.index.month

In [60]:
monthly = monthly.reset_index()

In [62]:
monthly.to_json('static_data/monthly_spending.json')